# report

In [187]:
import pandas as pd
import os

In [83]:
file_list=['spy/'+i for i in os.listdir('spy')]

In [84]:
sample = pd.read_csv(file_list[0],skipinitialspace=True)

In [91]:
import pandas as pd
import os

def contract_selct(
    selct_type: str = 'main',
    contract_type: str = 'C',
    data : str = 'spy',
    date: str = None,
    main: pd.DataFrame = None,
    maxDTE: int = 20,
    minDTE: int = 10,
    maxDistance: float = None,
    minDistance: float = None,
    maxTheta: float = None,
    minTheta: float = None,
    moneyness : str = 'out') ->pd.DataFrame:
    
"""
return a dataframe containing the information of selected contracts

input:
selct_type: "main" or "hedge", declare what type of contracts we are searching, whether the main contracts we are shorting, or the contracts that we used to hedge
contract_type: 'C' or 'P'
date: str in form of "yyyymmdd", declare the date
data : 'spy' or 'spx'
main: None when selct_type = 'main', a dataframe containing the main contract when selct_type = 'hedge'
maxDTE: max DTE
minDTE: min DTE
maxDistance: max distance of strike from underlying in percentage
minDistance: min distance of strike from underlying in percentage
maxTheta: max Theta
minTheta: min Theta
moneyness: 'out' or 'in'
"""
    contract_pd = pd.Dataframe()
    
    if selct_type == 'main':
        contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                           index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
        contract_pd = contract_pd.drop(columns='Unnamed: 0')
        contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
        contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]
        if contract_type == 'C':
            contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
            if moneyness == 'out':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
            if moneyness == 'in':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
        if contract_type == 'P':
            contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
            if moneyness == 'out':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
            if moneyness == 'in':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]            

    if selct_type == 'hedge':
        contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                           index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
        contract_pd = contract_pd.drop(columns='Unnamed: 0')
        contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
        contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]
        if contract_type == 'C':
            contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
            if moneyness == 'out':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
            if moneyness == 'in':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
        if contract_type == 'P':
            contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
            if moneyness == 'out':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
            if moneyness == 'in':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]            

In [245]:
data= 'spx'
date= '20200910'
maxDTE: int = 30,
minDTE: int = 10,
maxDistance: float = 0.2,
minDistance: float = 0.01,
maxTheta: float = -0.0,
minTheta: float = -1.2,
contract_type: str = 'P'
moneyness = 'out'

In [247]:
contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                   index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
contract_pd = contract_pd.drop(columns='Unnamed: 0')
contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]

In [241]:
contract_pd

[UNDERLYING_LAST]  [DTE]  [C_DELTA]  [C_GAMMA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                   
2020-09-10   2020-09-21                3337.5   11.0    0.96417    0.00016   
             2020-09-21                3337.5   11.0    0.96547    0.00023   
             2020-09-21                3337.5   11.0    0.96436    0.00017   
             2020-09-21                3337.5   11.0    0.96153    0.00023   
             2020-09-21                3337.5   11.0    0.95879    0.00022   
...                                       ...    ...        ...        ...   
             2020-10-09                3337.5   29.0    0.01231    0.00017   
             2020-10-09                3337.5   29.0    0.01058    0.00015   
             2020-10-09                3337.5   29.0    0.00885    0.00014   
             2020-10-09                3337.5   29.0    0.00796    0.00005   
             2020-10-09                3337.5   29.0    0.00531    0.00005   

                            [C_VEGA]  [C_THETA]  [C_BID]  [C_ASK]  [STRIKE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                    
2020-09-10   2020-09-21      0.47195   -1.25820   640.20   646.70    2700.0   
             2020-09-21      0.46332   -1.17310   615.69   620.30    2725.0   
             2020-09-21      0.47913   -1.17078   589.60   596.50    2750.0   
             2020-09-21      0.50866   -1.20929   566.01   570.70    2775.0   
             2020-09-21      0.52863   -1.21332   540.00   546.89    2800.0   
...                              ...        ...      ...      ...       ...   
             2020-10-09      0.30547   -0.11089     0.79     1.00    3875.0   
             2020-10-09      0.26183   -0.09547     0.65     0.85    3900.0   
             2020-10-09      0.23103   -0.08614     0.55     0.75    3925.0   
             2020-10-09      0.20053   -0.07487     0.45     0.66    3950.0   
             2020-10-09      0.15200   -0.05806     0.30     0.50    4000.0   

                            [P_BID]  [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                     
2020-09-10   2020-09-21        1.15     1.30   -0.01123    0.00009   0.17597   
             2020-09-21        1.30     1.56   -0.01255    0.00011   0.20088   
             2020-09-21        1.45     1.65   -0.01396    0.00009   0.21910   
             2020-09-21        1.65     1.89   -0.01587    0.00016   0.24735   
             2020-09-21        1.95     2.20   -0.01948    0.00018   0.28311   
...                             ...      ...        ...        ...       ...   
             2020-10-09      533.30   538.80   -1.00000    0.00000  14.14468   
             2020-10-09      558.29   563.20   -1.00000    0.00000  14.14268   
             2020-10-09      581.00   592.61   -1.00000    0.00000  14.14093   
             2020-10-09      604.00   617.00   -1.00000    0.00000  14.13886   
             2020-10-09      657.40   662.90   -1.00000    0.00000  14.13478   

                            [P_THETA]  [STRIKE_DISTANCE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                 
2020-09-10   2020-09-21      -0.34841              637.5   
             2020-09-21      -0.39243              612.5   
             2020-09-21      -0.41792              587.5   
             2020-09-21      -0.46382              562.5   
             2020-09-21      -0.52352              537.5   
...                               ...                ...   
             2020-10-09       0.00000              537.5   
             2020-10-09       0.00000              562.5   
             2020-10-09       0.00000              587.5   
             2020-10-09       0.00000              612.5   
             2020-10-09       0.00000              662.5   

                            [STRIKE_DISTANCE_PCT]  
[QUOTE_DATE] [EXPIRE_DATE]                         
2020-09-10   2020-09-21                     0.191  
       

In [248]:
if contract_type == 'C':
    contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
    if moneyness == 'out':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
    if moneyness == 'in':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
if contract_type == 'P':
    contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
    if moneyness == 'out':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
    if moneyness == 'in':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]

In [249]:
contract_pd

[UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                
2020-09-10   2020-09-21                3337.5   11.0    2700.0     1.15   
             2020-09-21                3337.5   11.0    2725.0     1.30   
             2020-09-21                3337.5   11.0    2750.0     1.45   
             2020-09-21                3337.5   11.0    2775.0     1.65   
             2020-09-21                3337.5   11.0    2800.0     1.95   
...                                       ...    ...       ...      ...   
             2020-10-09                3337.5   29.0    2990.0    20.60   
             2020-10-09                3337.5   29.0    3000.0    21.50   
             2020-10-09                3337.5   29.0    3010.0    22.30   
             2020-10-09                3337.5   29.0    3020.0    23.20   
             2020-10-09                3337.5   29.0    3025.0    23.70   

                            [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                            
2020-09-10   2020-09-21        1.30   -0.01123    0.00009   0.17597   
             2020-09-21        1.56   -0.01255    0.00011   0.20088   
             2020-09-21        1.65   -0.01396    0.00009   0.21910   
             2020-09-21        1.89   -0.01587    0.00016   0.24735   
             2020-09-21        2.20   -0.01948    0.00018   0.28311   
...                             ...        ...        ...       ...   
             2020-10-09       21.29   -0.12319    0.00058   1.95175   
             2020-10-09       22.29   -0.12849    0.00064   2.00962   
             2020-10-09       23.00   -0.13342    0.00072   2.06195   
             2020-10-09       23.99   -0.13898    0.00073   2.11941   
             2020-10-09       24.39   -0.14105    0.00074   2.14769   

                            [P_THETA]  [STRIKE_DISTANCE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                 
2020-09-10   2020-09-21      -0.34841              637.5   
             2020-09-21      -0.39243              612.5   
             2020-09-21      -0.41792              587.5   
             2020-09-21      -0.46382              562.5   
             2020-09-21      -0.52352              537.5   
...                               ...                ...   
             2020-10-09      -1.12062              347.5   
             2020-10-09      -1.14683              337.5   
             2020-10-09      -1.16410              327.5   
             2020-10-09      -1.18683              317.5   
             2020-10-09      -1.19631              312.5   

                            [STRIKE_DISTANCE_PCT]  
[QUOTE_DATE] [EXPIRE_DATE]                         
2020-09-10   2020-09-21                     0.191  
             2020-09-21                     0.184  
             2020-09-21                     0.176  
             2020-09-21                     0.169  
             2020-09-21                     0.161  
...                                           ...  
             2020-10-09                     0.104  
             2020-10-09                     0.101  
             2020-10-09                     0.098  
             2020-10-09                     0.095  
             2020-10-09                     0.094  

[264 rows x 11 columns]

In [250]:
contract_pd['spread_pct'] = (contract_pd['[P_ASK]']-contract_pd['[P_BID]'])*2/(contract_pd['[P_BID]']+contract_pd['[P_ASK]'])

In [251]:
for i in ['[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]']:
    contract_pd['price_to_'+i]=(contract_pd['[P_BID]']+contract_pd['[P_ASK]'])/(2*contract_pd[i])

In [258]:
contract_pd['utility'] = 5000*contract_pd['spread_pct'] + \
0.5*contract_pd['price_to_[P_DELTA]'] -0.005*contract_pd['price_to_[P_GAMMA]'] - \
contract_pd['price_to_[P_VEGA]'] -5* contract_pd['price_to_[P_THETA]']

In [259]:
contract_pd.sort_values(by=['utility'])

[UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                
2020-09-10   2020-10-09                3337.5   29.0    2940.0    17.51   
             2020-09-30                3337.5   20.0    2875.0     8.10   
             2020-10-09                3337.5   29.0    2850.0    12.10   
             2020-09-30                3337.5   20.0    2880.0     8.30   
             2020-09-30                3337.5   20.0    2985.0    12.80   
...                                       ...    ...       ...      ...   
             2020-09-21                3337.5   11.0    2800.0     1.95   
             2020-09-21                3337.5   11.0    2750.0     1.45   
             2020-09-21                3337.5   11.0    2775.0     1.65   
             2020-09-21                3337.5   11.0    2725.0     1.30   
             2020-09-23                3337.5   13.0    2980.0     6.00   

                            [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                            
2020-09-10   2020-10-09       18.00   -0.10291    0.00048   1.71489   
             2020-09-30        8.31   -0.05677    0.00030   0.91736   
             2020-10-09       12.50   -0.07161    0.00034   1.30393   
             2020-09-30        8.50   -0.05829    0.00035   0.93483   
             2020-09-30       13.11   -0.09249    0.00053   1.32768   
...                             ...        ...        ...       ...   
             2020-09-21        2.20   -0.01948    0.00018   0.28311   
             2020-09-21        1.65   -0.01396    0.00009   0.21910   
             2020-09-21        1.89   -0.01587    0.00016   0.24735   
             2020-09-21        1.56   -0.01255    0.00011   0.20088   
             2020-09-23        8.19   -0.06251    0.00052   0.80853   

                            [P_THETA]  [STRIKE_DISTANCE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                 
2020-09-10   2020-10-09      -1.03590              397.5   
             2020-09-30      -0.86969              462.5   
             2020-10-09      -0.84014              487.5   
             2020-09-30      -0.88301              457.5   
             2020-09-30      -1.13512              352.5   
...                               ...                ...   
             2020-09-21      -0.52352              537.5   
             2020-09-21      -0.41792              587.5   
             2020-09-21      -0.46382              562.5   
             2020-09-21      -0.39243              612.5   
             2020-09-23      -1.08945              357.5   

                            [STRIKE_DISTANCE_PCT]  spread_pct  \
[QUOTE_DATE] [EXPIRE_DATE]                                      
2020-09-10   2020-10-09                     0.119    0.027598   
             2020-09-30                     0.139    0.025594   
             2020-10-09                     0.146    0.032520   
             2020-09-30                     0.137    0.023810   
             2020-09-30                     0.106    0.023929   
...                                           ...         ...   
             2020-09-21                     0.161    0.120482   
             2020-09-21                     0.176    0.129032   
             2020-09-21                     0.169    0.135593   
             2020-09-21                     0.184    0.181818   
             2020-09-23                     0.107    0.308668   

                            price_to_[P_DELTA]  price_to_[P_GAMMA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                           
2020-09-10   2020-10-09            -172.529395        36989.583333   
             2020-09-30            -144.530562        27350.000000   
             2020-10-09            -171.763720        36176.470588   
             2020-09-30            -144.107051        24000.000000   
             2020-09-30            -140.069197        24443.396226   
...        